In [16]:
import cv2
import os
import shutil
import numpy as np
import zipfile
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tqdm import tqdm
from PIL import Image, ImageFilter, ImageEnhance
from keras.optimizers import Adam
from keras.regularizers import l2

In [2]:
# Unzipping the image dataset
zip_ref = zipfile.ZipFile('original.zip', 'r')
zip_ref.extractall('dataset')  # Extracting to the 'dataset' folder
zip_ref.close()

In [ ]:
def apply_fisheye(image):
    width, height = image.size
    radius = min(width, height) / 2
    center_x, center_y = width / 2, height / 2
    fisheye_image = image.copy()

    pixels = fisheye_image.load()
    original_pixels = image.load()

    for i in range(width):
        for j in range(height):
            rel_x = (i - center_x) / radius
            rel_y = (j - center_y) / radius

            original_distance = np.sqrt(rel_x**2 + rel_y**2)
            if original_distance == 0:
                continue

            new_distance = original_distance ** 2

            new_x = int(rel_x * new_distance * radius + center_x)
            new_y = int(rel_y * new_distance * radius + center_y)

            if 0 <= new_x < width and 0 <= new_y < height:
                pixels[i, j] = original_pixels[new_x, new_y]
            else:
                pixels[i, j] = (0, 0, 0)

    return fisheye_image

# Ensure the target directory exists
fisheye_dir = 'dataset/fisheye'
if not os.path.exists(fisheye_dir):
    os.makedirs(fisheye_dir)

# Specify the path to the original images
original_dir = 'dataset/original'
total_images = 20

# Create 10 fisheye images and save
for i in range(1, total_images + 1):
    img_path = f'{original_dir}/{i:02d}.jpg'
    img = Image.open(img_path).convert('RGB')
    for j in range(10):
        fisheye_img = apply_fisheye(img)
        fisheye_img.save(f'{fisheye_dir}/fisheye_{i:02d}_{j}.jpg')

In [7]:
# Define data paths
original_dir = 'dataset/original'
fisheye_dir = 'dataset/fisheye'
train_dir = 'dataset/train'
validation_dir = 'dataset/validation'

# Create directories for training and validation
for category in ['original', 'fisheye']:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, category), exist_ok=True)

# Function to split data and move files
def split_data(source_folder, train_folder, validation_folder, validation_split=0.2):
    files = os.listdir(source_folder)
    train_files, validation_files = train_test_split(files, test_size=validation_split)

    # Copy files to training directory
    for file in train_files:
        shutil.copy(os.path.join(source_folder, file), os.path.join(train_folder, file))

    # Copy files to validation directory
    for file in validation_files:
        shutil.copy(os.path.join(source_folder, file), os.path.join(validation_folder, file))

# Apply function to original and distorted datasets
split_data(original_dir, os.path.join(train_dir, 'original'), os.path.join(validation_dir, 'original'))
split_data(fisheye_dir, os.path.join(train_dir, 'fisheye'), os.path.join(validation_dir, 'fisheye'))

In [8]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(224, 224),
    batch_size=5,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(
    'dataset/validation',
    target_size=(224, 224),
    batch_size=5,
    class_mode='categorical'
)

Found 164 images belonging to 2 classes.
Found 41 images belonging to 2 classes.


In [9]:
import shutil

def remove_unwanted_dirs(directory):
    checkpoint_dir = os.path.join(directory, '.ipynb_checkpoints')
    if os.path.exists(checkpoint_dir):
        shutil.rmtree(checkpoint_dir)
        print(f"Removed {checkpoint_dir}")

remove_unwanted_dirs(train_dir)
remove_unwanted_dirs(validation_dir)

print("Updated training directories:", os.listdir(train_dir))
print("Updated validation directories:", os.listdir(validation_dir))

Updated training directories: ['original', 'fisheye']
Updated validation directories: ['original', 'fisheye']


In [10]:
import shutil
import os

directories_to_remove = ['original', 'fisheye']
base_dir = 'dataset'

for dir_name in directories_to_remove:
    dir_path = os.path.join(base_dir, dir_name)
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
        print(f"Removed directory: {dir_path}")
    else:
        print(f"Directory does not exist, no need to remove: {dir_path}")

print("Updated contents of the dataset directory:", os.listdir(base_dir))

Removed directory: dataset/original
Removed directory: dataset/fisheye
Updated contents of the dataset directory: ['validation', 'train']


In [17]:
!pip install tensorflow_addons
import tensorflow_addons as tfa

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)  # L2
x = Dropout(0.5)(x)  # Dropout
predictions = Dense(1, activation='sigmoid')(x)

optimizer = tfa.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.001)

# Compile the model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Prepare training data
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2  # Use 20% of the data as validation set
)

train_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=5,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=5,
    class_mode='binary',
    subset='validation'
)

# Use early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)

Found 165 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [18]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=15,
    epochs=10,
    callbacks=[early_stopping],
    validation_data=validation_generator,
    validation_steps=5
)

# Evaluate the model
results = model.evaluate(validation_generator)
print("Test loss, Test accuracy:", results)

Epoch 1/10
15/15 [==============================] - 71s 5s/step - loss: 6.8388 - accuracy: 0.6933 - val_loss: 8.0077 - val_accuracy: 0.3200
Epoch 2/10
15/15 [==============================] - 58s 4s/step - loss: 7.4211 - accuracy: 0.7600 - val_loss: 21.9727 - val_accuracy: 0.3600
Epoch 3/10
15/15 [==============================] - 58s 4s/step - loss: 6.4696 - accuracy: 0.7733 - val_loss: 20.9747 - val_accuracy: 0.3200
Epoch 4/10
15/15 [==============================] - 57s 4s/step - loss: 6.0537 - accuracy: 0.7600 - val_loss: 11.1107 - val_accuracy: 0.3600
Epoch 5/10
15/15 [==============================] - 63s 4s/step - loss: 5.4660 - accuracy: 0.8000 - val_loss: 10.6525 - val_accuracy: 0.4400
Epoch 6/10
15/15 [==============================] - 56s 4s/step - loss: 4.6981 - accuracy: 0.7600 - val_loss: 22.3181 - val_accuracy: 0.3600
Epoch 7/10
15/15 [==============================] - 57s 4s/step - loss: 8.4083 - accuracy: 0.7067 - val_loss: 32.3394 - val_accuracy: 0.2800
Epoch 8/10
15/